<a href="https://colab.research.google.com/github/pinilDissanayaka/Google-Palm2-RAG-application/blob/main/QA_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip uninstall sentence-transformers
!pip install sentence-transformers==2.2.2
!pip install faiss-gpu
!pip install langchain
!pip install langchain_community

Found existing installation: sentence-transformers 2.2.2
Uninstalling sentence-transformers-2.2.2:
  Would remove:
    /usr/local/lib/python3.10/dist-packages/sentence_transformers-2.2.2.dist-info/*
    /usr/local/lib/python3.10/dist-packages/sentence_transformers/*
Proceed (Y/n)? Y
  Successfully uninstalled sentence-transformers-2.2.2
  Using cached sentence_transformers-2.2.2-py3-none-any.whl
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 5.0 MB/s eta 0:00:00


In [11]:
import os
from langchain.llms import GooglePalm
from langchain.document_loaders.csv_loader import CSVLoader
from langchain.embeddings import HuggingFaceInstructEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

In [12]:
from google.colab import userdata
os.environ['GOOGLE_API_KEY']=userdata.get('GOOGLE_API_KEY')

In [13]:
llm=GooglePalm(google_api_key=GOOGLE_API_KEY)

In [ ]:
loader=CSVLoader(file_path="/content/codebasics_faqs.csv", source_column='prompt', encoding="latin-1")
data=loader.load()

In [ ]:
model_name = "hkunlp/instructor-large"
embeddingModel = HuggingFaceInstructEmbeddings(
    query_instruction="Represent the query for retrieval: ",
    model_name=model_name
)

load INSTRUCTOR_Transformer
max_seq_length  512


In [ ]:
vectorDAtabase=FAISS.from_documents(data, embeddingModel)

In [ ]:
retriever=vectorDAtabase.as_retriever()

In [ ]:
retriever.get_relevant_documents("How to install python")

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


[Document(page_content='prompt: Is there any prerequisite for taking this bootcamp ?\nresponse: Our bootcamp is specifically designed for beginners with no prior experience in this field. The only prerequisite is that you need to have a functional laptop with at least 4GB ram, an internet connection, and a thrill to learn data analysis.', metadata={'source': 'Is there any prerequisite for taking this bootcamp ?', 'row': 2}),
 Document(page_content='prompt: Does Power BI work in Mac OS/Ubuntu?\nresponse: Power BI desktop works only in Windows OS. Please look into the system requirements section on this page. However, you can use a virtual machine to install and work with Power BI in other Operating systems.', metadata={'source': 'Does Power BI work in Mac OS/Ubuntu?', 'row': 31}),
 Document(page_content='prompt: Is there any prerequisite for taking this course?\nresponse: The only prerequisite is that you need to have a functional laptop with at least 4GB ram, internet connection and a 

In [ ]:
prompt_template = """Given the following context and a question, generate an answer based on this context only.
In the answer try to provide as much text as possible from "response" section in the source document context without making much changes.
If the answer is not found in the context, kindly state "I don't know." Don't try to make up an answer.

CONTEXT: {context}

QUESTION: {question}"""


PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)
chain_type_kwargs = {"prompt": PROMPT}

chain = RetrievalQA.from_chain_type(llm=llm,
                            chain_type="stuff",
                            retriever=retriever,
                            input_key="query",
                            return_source_documents=True,
                            chain_type_kwargs=chain_type_kwargs)

In [ ]:
chain('Do you provide job assistance and also do you provide job gurantee?')

{'query': 'Do you provide job assistance and also do you provide job gurantee?',
 'result': 'Yes, We help you with resume and interview preparation along with that we help you in building online credibility, and based on requirements we refer candidates to potential recruiters. We created a much lighter version of this course on YouTube available for free (click this link) and many people gave us feedback that they were able to fetch jobs (see testimonials). Now this paid course is at least 5x better than the YouTube course which gives us ample confidence that you will be able to get a job. However, we want to be honest and do not want to make any impractical promises! Our guarantee is to prepare you for the job market by teaching the most relevant skills, knowledge & timeless principles good enough to fetch the job.',
 'source_documents': [Document(page_content='prompt: Do you provide any job assistance?\nresponse: Yes, We help you with resume and interview preparation along with that